In [1]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
#Run the below piece of code for the first time
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/alvaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
dialogs_data = pd.read_csv("dialog_lines.csv",encoding = "UTF8")
dialogs_data.head()

,racist/not_racist,dialog_line
0,not_racist,\n\n\n\n\n
1,not_racist,GRAN TORINO\n\n\n\n ...
2,not_racist,FADE IN\n\n\n\n INT CHURCH\n\n\n ...
3,not_racist,\n\n\n OLDER MAN\n\n Im real...
4,not_racist,Walt She was a real peach\n\n\n


In [5]:
dialogs_data.groupby('racist/not_racist').describe()


dialog_line         \
                        count unique   
racist/not_racist                      
not_racist              38514  32771   
racist                    877    877   

                                                                           
                                                                 top freq  
racist/not_racist                                                          
not_racist                                   \n\n                     304  
racist             \n\n\n\n\n          \n\n\n\n          \n\n    ...    1

In [6]:
dialogs_data_copy = dialogs_data['dialog_line'].copy()


In [7]:
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [8]:
dialogs_data_copy = dialogs_data_copy.apply(text_preprocess)

In [9]:
dialogs_data_copy

0                                                         
1                          GRAN TORINO Written Nick Schenk
2                                          FADE INT CHURCH
3                          OLDER MAN Im real sorry Dorothy
4                                          Walt real peach
                               ...                        
39386                                Thanks clothes RICHIE
39387    Youre welcome Frank glances away three young h...
39388    Uhoh Look come gangsters Franks right path doe...
39389    first one still staring Frank finally good sen...
39390                 jacket props collar points direction
Name: dialog_line, Length: 39391, dtype: object

In [10]:
vectorizer = TfidfVectorizer()

In [11]:
dialogs_mat = vectorizer.fit_transform(dialogs_data_copy)
dialogs_mat

<39391x23573 sparse matrix of type '<class 'numpy.float64'>'
	with 291287 stored elements in Compressed Sparse Row format>

In [16]:
dialogs_train, dialogs_test, r_nr_train, r_nr_test = train_test_split(dialogs_mat, 
                                                        dialogs_data['racist/not_racist'], test_size=0.3, random_state=20)


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

racist_model = LogisticRegression(solver='liblinear', penalty='l1')
racist_model.fit(dialogs_train, r_nr_train)
pred = racist_model.predict(dialogs_test)
accuracy_score(r_nr_test,pred)

0.9812150956168556